반복문을 이용한 데이터 수집

In [ ]:
import requests
import json
import pandas as pd
from datasets import Dataset
import huggingface_hub

In [ ]:
file_path = "TL_span_extraction.json"
try:
    # 파일 열고 내용 읽기
    with open(file_path, "r", encoding="utf-8-sig") as f:  # 적절한 인코딩 설정
        original_data = json.load(f)
        print(original_data)

except Exception as e:
    print(e)

In [ ]:
# 원본데이터의 키 확인하기
original_data.keys()

In [ ]:
len( original_data["data"] )

In [ ]:
datas = original_data["data"]
datas

In [ ]:
# 파인튜닝을 위한 데이터 수집 리스트
alpacaData = []
columnList = []

# RAG를 위한 원본 데이터 수집 리스트
contextData = []
columnList2 = []

In [ ]:
# 수집해야 하는 데이터
targetClass = "법률"

In [ ]:
for data in datas:
    # 하나의 데이터 셋 내 주요 파라그래프 키 확인하기
    data.keys()

    if(targetClass in data["doc_class"]["class"]):
        columnList2=[]
        columnList2.append(data["doc_id"])
        columnList2.append(data["doc_title"])

        for eachParagraph in data["paragraphs"]:
            columnList2.append(eachParagraph["context"])
            contextData.append(columnList2)

            # 딕셔너리 구조의 키 중 주요 정보 키값으로 가져오기
            qasDatas = eachParagraph["qas"]

            for qasData in qasDatas:
                columnList = []
                # 질문
                columnList.append(qasData["question"])

                # 정답
                columnList.append(qasData["answer"]["text"])

                # input
                columnList.append("")

                # class
                # columnList.append(data["doc_class"]["class"])

                alpacaData.append(columnList)


### instrunction : 질문
### output       : 정답
### input        : 없음
alpacaDf = pd.DataFrame(alpacaData)
alpacaDf.columns = ["instruction", "output", "input"]

contextDf = pd.DataFrame(contextData)
contextDf.columns = ["id", "title", "context"]
contextDf.to_csv("./context.csv", index=False)

In [ ]:
alpacaDf.head(5)

In [ ]:
print(len(alpacaData))
print(len(contextDf))

In [ ]:
# DataFrame에서 하나의 값만 보기
alpacaDf.loc[0,["instruction"]].values

In [ ]:
# prompt: 허깅페이스 로그인 코드생성

import os
from huggingface_hub import login

# Hugging Face Hub에 로그인합니다.
login(token="hugginggace_token")

In [ ]:
dataset = Dataset.from_pandas(alpacaDf)

In [ ]:
dataset.push_to_hub("limjh12/ko-law-alpaca")